In [1]:
# Author: M. Riley Owens (GitHub: mrileyowens)

# This file makes stacks of the spectra of the LyC-leaking 
# region and the non-LyC-leaking regions.

In [2]:
import os
import glob

import numpy as np

In [3]:
def stack():
    
    '''
    Stacks individual MagE spectra into two stacked spectra: one of the LyC-leaking region, and another 
    of the non-LyC-leaking regions
    '''

    def get_data(file):

        '''
        Extracts the wavelength, flux density, and uncertainty from a given MagE spectrum

        Parameters:
            file : str
                Complete file path to the MagE spectrum

        Returns:
            w : numpy.ndarray
                Observed wavelength bins of the spectrum (angstroms)
            f : numpy.ndarray
                Observed flux densities of the spectrum (erg/s/cm^2/angstrom)
            n : numpy.ndarray
                Gaussian 1σ observed flux density uncertainties of the spectrum
        '''

        # Get the observed wavelengths (angstroms), flux densities (erg/s/cm^2/Hz), and Gaussian 1σ flux density uncertainties of the spectrum
        w, f, n = np.loadtxt(file, delimiter='\t', comments=('#', 'w'), usecols=(0,1,2), unpack=True)

        # Only keep array entries where the flux densities and uncertainties are physically sensible
        w = w[f < 1e-20]
        n = n[f < 1e-20]
        f = f[f < 1e-20]

        w = w[n < 1e-20]
        f = f[n < 1e-20]
        n = n[n < 1e-20]

        # Convert the flux densities and uncertainties to wavelength space
        f = f * 2.998e18 / np.square(w)
        n = n * 2.998e18 / np.square(w)

        return w, f, n

    # Establish basic directories
    home = os.getcwd()
    data = f'{home}/data'

    # Get the file paths of the MagE spectra. This line is not optimal, since it will include
    # the stacked spectra if the notebook executes again after creating them, which could
    # cause problems later in the notebook. I (M. Riley Owens) am leaving it as is with this 
    # danger noted
    files = glob.glob(f'{data}/mage/*.txt')

    # Dictionary of the spectroscopic redshifts of each spectrum, as 
    # measured by Mainali et al. (2022) (ApJ, 940, 160) from the narrow 
    # component of the [O III] 5007 Angstrom nebular emission line.
    z_dict = {
        'sunburst_M-0-comb1_MWdr' : 2.37014,
        'sunburst_M-2-comb1_MWdr' : 2.37017,
        'sunburst_M-3-comb1_MWdr' : 2.37025,
        'sunburst_M-4-comb1_MWdr' : 2.37073,
        'sunburst_M-5-comb1_MWdr' : 2.37086,
        'sunburst_M-6-comb1_MWdr' : 2.37021,
        'sunburst_M-7-comb1_MWdr' : 2.37044,
        'sunburst_M-8-comb1_MWdr' : 2.37024,
        'sunburst_M-9-comb1_MWdr' : 2.37030
    }

    # Make empty arrays for the final stacked spectra,
    # which will be filled in later
    w_leaker = np.array([])
    f_leaker = np.array([])
    n_leaker = np.array([])

    w_nonleaker = np.array([])
    f_nonleaker = np.array([])
    n_nonleaker = np.array([])

    # For each individual spectrum
    for i, file in enumerate(files):

        # If the spectrum is one of the LyC leaker (excluding slit M0
        # because of poor observing conditions leading to a poor fluxing)
        if any(string in file for string in ['M-2','M-7','M-8','M-9']):

            # Get the observed wavelength (angstroms), flux densities (erg/s/cm^2/angstrom), 
            # and Gaussian 1σ flux density uncertainties of the MagE spectrum
            w, f, n = get_data(file)      

            # Place the wavelength bins in the rest frame
            w = w / (1 + z_dict[os.path.basename(file)[:-4]])

            # Normalize the flux densities and uncertainties by the median flux density
            # between rest-frame 1267 - 1276 angstroms, just like the continuum fits to
            # the individual MagE spectra created by John Chisholm
            n = n / np.median(f[(w >= 1267) & (w <= 1276)])
            f = f / np.median(f[(w >= 1267) & (w <= 1276)])

            # If the final arrays of the stacked LyC leaker spectrum have not
            # been instantiated yet, do so now
            if len(w_leaker) == 0:

                w_leaker = w
                f_leaker = f
                n_leaker = n

            else:

                # Interpolate the data at the wavelength bins of the stacked spectrum
                f = np.interp(w_leaker, w, f, left=0, right=0)
                n = np.interp(w_leaker, w, n, left=0, right=0)

                # Add the flux density and flux density uncertainties to the stacked spectrum
                # appropriately
                f_leaker += f
                n_leaker = np.sqrt((n_leaker)**2 + (n)**2)

        # If the spectrum is of a non-LyC-leaking region (excluding slit M3 because it captures a highly compact object; see Vanzella 
        # et al. (2020) (MNRAS, 499, L67), Diego et al. (2022) (A&A, 665, A134), or Sharon et al. (2022) (ApJ, 941, 203))
        elif any(string in file for string in ['M-5','M-4','M-6']):

            # Get the observed wavelength (angstroms), flux densities (erg/s/cm^2/angstrom), 
            # and Gaussian 1σ flux density uncertainties of the MagE spectrum
            w, f, n = get_data(file)

            # Place the wavelength bins in the rest frame
            w = w / (1 + z_dict[os.path.basename(file)[:-4]])

            # Normalize the flux densities and uncertainties by the median flux density
            # between rest-frame 1267 - 1276 angstroms, just like the continuum fits to
            # the individual MagE spectra created by John Chisholm
            n = n / np.median(f[(w >= 1267) & (w <= 1276)])
            f = f / np.median(f[(w >= 1267) & (w <= 1276)])

            # If the final arrays of the stacked non-LyC-leaking spectrum have not
            # been instantiated yet, do so now
            if len(w_nonleaker) == 0:

                w_nonleaker = w
                f_nonleaker = f
                n_nonleaker = n

            else:

                # Interpolate the data at the wavelength bins of the stacked spectrum
                f = np.interp(w_nonleaker, w, f, left=0, right=0)
                n = np.interp(w_nonleaker, w, n, left=0, right=0)

                # Add the flux density and flux density uncertainties to the stacked spectrum
                # appropriately
                f_nonleaker += f
                n_nonleaker = np.sqrt((n_nonleaker)**2 + (n)**2)

    # Normalize the two stacks by the number of composite spectra
    f_leaker = f_leaker / 4
    n_leaker = n_leaker / 4

    f_nonleaker = f_nonleaker / 3
    n_nonleaker = n_nonleaker / 3

    # Save the stacked spectra to .txt files
    np.savetxt(f'{data}/mage/sunburst_arc_leaker_stack_mage.txt', np.array([w_leaker, f_leaker, n_leaker]).T, delimiter='\t', encoding='utf-8',
        header=f'Stacked spectrum of the LyC-leaking region of the Sunburst Arc from the MagE slits M2, M7, M8, M9, created by stack.ipynb.' + '\n' 
        + 'The stack excludes slit M0 because of poor observing conditions that caused a poor fluxing.' + '\n' + '\n'
        + 'The columns below are, from left to right: rest wavelength (angstroms), flux density (arbitrary units in wavelength space), and the 1σ Gaussian uncertainty of the flux density.')
    np.savetxt(f'{data}/mage/sunburst_arc_nonleaker_stack_mage.txt', np.array([w_nonleaker, f_nonleaker, n_nonleaker]).T, delimiter='\t', encoding='utf-8',
        header=f'Stacked spectrum of the non-LyC-leaking regions of the Sunburst Arc from the MagE slits M5, M4, M6, created by stack.ipynb.' + '\n' 
        + 'The stack excludes slit M3 because it captures a highly compact region of the galaxy which may not be representative of it\'s surrounding environment.' + '\n'
        + 'Consult Vanzella et al. (2020) (MNRAS:Letters, 499, 1, L67), Diego et al. (2022) (A&A, 665, A134), or Sharon et al. (2022) (ApJ, 941, 2, 203) for more information about the compact region.' + '\n' + '\n'
        + 'The columns below are, from left to right: rest wavelength (angstroms), flux density (arbitrary units in wavelength space), and the 1σ Gaussian uncertainty of the flux density.')

In [4]:
stack()